In [1]:
# ==================== SETTING UP THE MODEL FROM OPEN AI ====================

import os
import json
import pandas as pd
from dotenv import load_dotenv
from langchain_openai import OpenAI
from openai import OpenAI

# ==================== LODING THE ENV VARIABLES =========================
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

# ==================== INISIATING THE CLIENT =============================
client = OpenAI()

In [2]:
from sqlalchemy import create_engine
import pandas as pd


# Path to your SQLite database file
database_file_path = "./db/test.db"

# Create an engine to connect to the SQLite database
# SQLite only requires the path to the database file
engine = create_engine(f'sqlite:///{database_file_path}')
file_url = "./data/all-states-history.csv"
df = pd.read_csv(file_url).fillna(value = 0)
df.to_sql(
    'all_states_history',
    con=engine,
    if_exists='replace',
    index=False
)

20780

In [3]:
# Creating the two function for the retival of the data from the database
import numpy as np
from sqlalchemy import text

def get_hospitalized_increase_for_state_on_date(state_abbr, specific_date):
    try:
        query = f"""
        SELECT date, hospitalizedIncrease
        FROM all_states_history
        WHERE state = '{state_abbr}' AND date = '{specific_date}';
        """
        query = text(query)

        with engine.connect() as connection:
            result = pd.read_sql_query(query, connection)
        if not result.empty:
            return result.to_dict('records')[0]
        else:
            return np.nan
    except Exception as e:
        print(e)
        return np.nan


def get_positive_cases_for_state_on_date(state_abbr, specific_date):
    try:
        query = f"""
        SELECT date, state, positiveIncrease AS positive_cases
        FROM all_states_history
        WHERE state = '{state_abbr}' AND date = '{specific_date}';
        """
        query = text(query)

        with engine.connect() as connection:
            result = pd.read_sql_query(query, connection)
        if not result.empty:
            return result.to_dict('records')[0]
        else:
            return np.nan
    except Exception as e:
        print(e)
        return np.nan


get_hospitalized_increase_for_state_on_date("AK","2021-03-05")

{'date': '2021-03-05', 'hospitalizedIncrease': 3}

In [4]:
tools_sql = [
    {
        "type": "function",
        "function": {
            "name": "get_hospitalized_increase_for_state_on_date",
            "description": """Retrieves the daily increase in
                              hospitalizations for a specific state
                              on a specific date.""",
            "parameters": {
                "type": "object",
                "properties": {
                    "state_abbr": {
                        "type": "string",
                        "description": """The abbreviation of the state
                                          (e.g., 'NY', 'CA')."""
                    },
                    "specific_date": {
                        "type": "string",
                        "description": """The specific date for
                                          the query in 'YYYY-MM-DD'
                                          format."""
                    }
                },
                "required": ["state_abbr", "specific_date"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_positive_cases_for_state_on_date",
            "description": """Retrieves the daily increase in 
                              positive cases for a specific state
                              on a specific date.""",
            "parameters": {
                "type": "object",
                "properties": {
                    "state_abbr": {
                        "type": "string",
                        "description": """The abbreviation of the 
                                          state (e.g., 'NY', 'CA')."""
                    },
                    "specific_date": {
                        "type": "string",
                        "description": """The specific date for the
                                          query in 'YYYY-MM-DD'
                                          format."""
                    }
                },
                "required": ["state_abbr", "specific_date"]
            }
        }
    }
]

In [5]:
# I) Create assistant
assistant = client.beta.assistants.create(
    name="Data Analyst",
    instructions="You are an assistant answering questions about a Covid dataset.",
    model="gpt-4o-mini-2024-07-18",
    tools=tools_sql
)


# II) Create thread
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_reg4Fvr0q2hU5oLKfeKqPOv9', created_at=1722446231, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [23]:
# III) Add message
# message = client.beta.threads.messages.create(
#     thread_id=thread.id,
#     role="user",
#     content="""how many hospitalized people we had in Alaska
#                the 2021-03-05?"""
# )

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="which state has more covid positve cases in July of 2021?"
)
print(message)

Message(id='msg_fAO3iMnCloSoj7BBJVEKAb1V', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='which state has more covid positve cases in July of 2021?'), type='text')], created_at=1722447895, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_reg4Fvr0q2hU5oLKfeKqPOv9')


In [24]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_fAO3iMnCloSoj7BBJVEKAb1V",
      "assistant_id": null,
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "which state has more covid positve cases in July of 2021?"
          },
          "type": "text"
        }
      ],
      "created_at": 1722447895,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "user",
      "run_id": null,
      "status": null,
      "thread_id": "thread_reg4Fvr0q2hU5oLKfeKqPOv9"
    },
    {
      "id": "msg_ses850NdpCgZZpwo34XOQ4Jx",
      "assistant_id": "asst_vcpvOkmTKooRnWncWtzzD35P",
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "I currently don't have direct access to a specific list of states from the Covid data you're referri

In [25]:
# IV) Run assistant on thread

run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

In [26]:
# Leverage the function calling with Assistant API
import time
from IPython.display import clear_output

start_time = time.time()

status = run.status

while status not in ["completed", "cancelled", "expired", "failed"]:
    time.sleep(5)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,run_id=run.id
    )
    print("Elapsed time: {} minutes {} seconds".format(
        int((time.time() - start_time) // 60),
        int((time.time() - start_time) % 60))
         )
    status = run.status
    print(f'Status: {status}')
    if (status=="requires_action"):
        available_functions = {
            "get_positive_cases_for_state_on_date": get_positive_cases_for_state_on_date,
            "get_hospitalized_increase_for_state_on_date":get_hospitalized_increase_for_state_on_date
        }

        tool_outputs = []
        for tool_call in run.required_action.submit_tool_outputs.tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                state_abbr=function_args.get("state_abbr"),
                specific_date=function_args.get("specific_date"),
            )
            print(function_response)
            print(tool_call.id)
            tool_outputs.append(
                { "tool_call_id": tool_call.id,
                 "output": str(function_response)
                }
            )

        run = client.beta.threads.runs.submit_tool_outputs(
          thread_id=thread.id,
          run_id=run.id,
          tool_outputs = tool_outputs
        )


messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

Elapsed time: 0 minutes 5 seconds
Status: completed
{
  "data": [
    {
      "id": "msg_e7N8rkcF8rZ3tgJIR00Juf0d",
      "assistant_id": "asst_vcpvOkmTKooRnWncWtzzD35P",
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "To identify which state had the most Covid-19 positive cases in July 2021, I would need to analyze the positive case counts for each state during that month. Unfortunately, I currently don't have access to the detailed data required to answer that.\n\nHowever, states like California, Texas, and Florida generally reported high numbers of positive cases during that period. If you need information for a specific state or date, please let me know!"
          },
          "type": "text"
        }
      ],
      "created_at": 1722447903,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "as

In [30]:
file = client.files.create(
  file=open("./data/all-states-history.csv", "rb"),
  purpose='assistants'
)

assistant = client.beta.assistants.create(
  instructions="""You are an assitant answering questions about
                  a Covid dataset.""",
  model="gpt-4o-mini-2024-07-18", 
  tools=[{"type": "code_interpreter"}],
  tool_resources={"code_interpreter": {"file_ids": [file.id]}}
)

thread = client.beta.threads.create()
print(thread)

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="In which state we have more postive cases on July 2021?"
)
print(message)

run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

Thread(id='thread_G3pydfHnK23uAw5Ix1zBHL4R', created_at=1722450957, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))
Message(id='msg_SJiF3bpznFncHScqdfNCHZs2', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='In which state we have more postive cases on July 2021?'), type='text')], created_at=1722450958, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_G3pydfHnK23uAw5Ix1zBHL4R')


In [31]:
status = run.status
start_time = time.time()
while status not in ["completed", "cancelled", "expired", "failed"]:
    time.sleep(5)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    print("Elapsed time: {} minutes {} seconds".format(
        int((time.time() - start_time) // 60),
        int((time.time() - start_time) % 60))
         )
    status = run.status
    print(f'Status: {status}')
    clear_output(wait=True)


messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_Y7X76zcXdo2ZAURinaCkpfn0",
      "assistant_id": "asst_pKw21ozptVHiw9gs8fACd8TH",
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "The filtered dataset for July 2021 is empty, indicating that there are no recorded positive cases for that month in the dataset. \n\nIf you need information for another date range or additional details, please let me know!"
          },
          "type": "text"
        }
      ],
      "created_at": 1722450984,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_5rwIXF6cYZHySdFgVAnj9ZYf",
      "status": null,
      "thread_id": "thread_G3pydfHnK23uAw5Ix1zBHL4R"
    },
    {
      "id": "msg_smEzkYN777vxwpKT2RQleXD5",
      "assistant_id": "asst_pKw21ozptVHiw9gs8fACd8TH",
      "attachments": [],
      "